In [38]:
cd /home/nakedmoleratvoices/Mole\ rats\ reborn/CodeRefactoredFinal/

/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal


In [39]:
import pandas as pd
import numpy as np
import pickle
import os
import tensorflow as tf
from skimage.transform import resize
import librosa
import matplotlib.pyplot as plt
from tracing_model.segmentator_training import predict_with_augmentation
from process_new_data.metadata_processing import compute_mask
import re
import shutil
%matplotlib inline

In [40]:
def what_sound_rec_name(sound_name,):
    s, e = sound_name.find('_'), sound_name.rfind('_')
    rec_name = sound_name[s + 1 : e] + '.npy'
    return rec_name

def update_trace_name(tn):

    new_tn = tn.replace('unconf', 'fresh')

    if 'freshtrace' in new_tn:
        timestamps = re.findall('[0-9]+\.[0-9]+\-[0-9]+\.[0-9]+', new_tn)[0]
        new_tn = (new_tn[:10] + '_' + new_tn[10:new_tn.find('_')] + '_' + timestamps.replace('-', '_') + 
             new_tn[10 + new_tn[10:].find('_') : ].replace('_' + timestamps, ''))

    else:
        timestamps = re.findall('[0-9]+\.[0-9]+\-[0-9]+\.[0-9]+', new_tn)[0]
        new_tn = (new_tn[:5] + '_' + new_tn[5:new_tn.find('_')] + '_' + timestamps.replace('-', '_') + 
             new_tn[5 + new_tn[5:].find('_') : ].replace('_' + timestamps, ''))
    return new_tn
    

In [41]:
# path to metadata
path_to_recordings_metadata = '/home/nakedmoleratvoices/Mole rats reborn/CodeRefactoredFinal/data/'
recordings_metadata_name = 'recordings_metadata.csv'
recs_metadata = pd.read_csv(path_to_recordings_metadata + recordings_metadata_name, )
recs_metadata.head()

,colony,date,experiment,name,number,path,processing stage,ratids
0,baratheon,21-06-19,single rat softchirps,baratheon_21-06-19_9449_0000001.npy,1,/home/nakedmoleratvoices/Mole rats reborn/Code...,traced and checked,9449
1,martell,08-11-19,single rat softchirps,martell_08-11-19_9438_0000035.npy,35,/home/nakedmoleratvoices/Mole rats reborn/Code...,traced and checked,9438
2,dothrakia,21-08-19,single rat softchirps,dothrakia_21-08-19_3006_0000008.npy,8,/home/nakedmoleratvoices/Mole rats reborn/Code...,traced and checked,3006
3,targaryen,21-06-19,single rat softchirps,targaryen_21-06-19_1540_0000001.npy,1,/home/nakedmoleratvoices/Mole rats reborn/Code...,traced and checked,1540
4,stark,21-06-19,single rat softchirps,stark_21-06-19_1545_0000001.npy,1,/home/nakedmoleratvoices/Mole rats reborn/Code...,traced and checked,1545


In [32]:
path_to_old_traces = '/home/nakedmoleratvoices/Mole rats reborn/Playground/queendata/traces/'
old_names = os.listdir(path_to_old_traces)
recs_to_update = set()
recs_not_found = set()
for tn in old_names:
    rec_name = what_sound_rec_name(tn)
    old_rec_path = path_to_old_traces.replace('traces', 'recordings')
    if rec_name not in recs_metadata['name'].values:
        recs_not_found.add(rec_name)
        continue
    rec_ind = np.where(recs_metadata['name'] == rec_name)[0][0]
    rec_path = recs_metadata[recs_metadata['name'] == rec_name]['path'].values[0]
    
    new_tn = update_trace_name(tn)
    recs_to_update.add(rec_name)
    shutil.copy(src = path_to_old_traces + tn, dst = rec_path.replace('recordings', 'traces') + new_tn)
    
for rec_name in recs_to_update:
    rec_ind = np.where(recs_metadata['name'] == rec_name)[0][0]
    recs_metadata['processing stage'].iloc[rec_ind] = 'traced and checked'
recs_metadata.to_csv(path_to_recordings_metadata + recordings_metadata_name, index =  None)